In [ ]:
import os
#os.chdir("../") #RUN ONCE  
%pwd


'c:\\Users\\FASI OWAIZ AHMED\\Desktop\\VS Code\\end_to_end_1'

In [14]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass 
class DataIngestconfig:   #all these parameters are matching config.yaml data_ingestion
    root_dir: Path 
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path

In [21]:
from src.datascience.constants import *
from src.datascience.utils.common import create_directories, read_yaml #i cant import *
# You might need this if read_yaml returns a Box object for dot notation access
# from box import ConfigBox

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH, #defined in constant
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
                 #schema_filepath=Path("schema.yaml")): # Comment kept for context

        # Read the YAML files and store their content
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Now self.config holds the content of config.yaml
        # Ensure 'artifacts_root' is a top-level key in your config.yaml
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestconfig:
        # This part should now work correctly as self.config holds the loaded data
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestconfig(
            root_dir=config.root_dir,  #since its config we can access it using .key
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir # Use the corrected name

        )
        return data_ingestion_config


In [28]:
#data ingestion component?
import urllib.request as request
#from src.datascience import logger  #logger?? where is it??
import logging as logger
import zipfile


class DataIngestion:
    def __init__(self, config:DataIngestconfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with info:\n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [29]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e